<a href="https://colab.research.google.com/github/Parth-Ach2002/Image-Classification/blob/main/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importing the necessary libraries**

In [ ]:
import tensorflow as tf
import keras
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt

**Connecting google drive with our file in order to access the contents of the dataset**

In [ ]:
# Mounting the Data from Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Setting the path for all the datasets**

In [ ]:
# Set path for all three datasets
train_path="/content/drive/MyDrive/Image Classification Dataset/Train"
test_path="/content/drive/MyDrive/Image Classification Dataset/Test"
val_path="/content/drive/MyDrive/Image Classification Dataset/Validation"

**Generating image data along with few transformations**

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255, rotation_range=30, width_shift_range=10.0, height_shift_range=10.0, shear_range=10,
                                   zoom_range=0.1,
                                   vertical_flip=True)
test_datagen = ImageDataGenerator(rescale = 1./255)
val_datagen = ImageDataGenerator(rescale = 1./255)


In [ ]:
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 16
                                                )
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (224, 224),
                                            batch_size = 16
                                            )

val_set = val_datagen.flow_from_directory(val_path,
                                            target_size = (224, 224),
                                            batch_size = 16
                                            )

Found 210 images belonging to 3 classes.
Found 60 images belonging to 3 classes.
Found 30 images belonging to 3 classes.


**Printing the class indices**

In [ ]:
training_set.class_indices

{'No Parking': 0, 'One Way': 1, 'Zebra Crossing': 2}

**Installing Keras Tuner for hyperparameter optimization**

In [ ]:
pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 2.4 MB/s eta 0:00:00


**Creating the model architecture**

In [16]:
def build_model(hp):
    model = models.Sequential()
    model.add(layers.Conv2D(hp.Int('conv1_units', min_value=32, max_value=256, step=32), (3, 3), activation='relu', input_shape=(224, 224, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(hp.Int('conv2_units', min_value=32, max_value=128, step=32), (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(hp.Float('dropout1', min_value=0.2, max_value=0.5, step=0.1)))
    model.add(layers.Conv2D(hp.Int('conv3_units', min_value=16, max_value=64, step=16), (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(hp.Int('dense1_units', min_value=32, max_value=128, step=32), activation='relu'))
    model.add(layers.Dropout(hp.Float('dropout2', min_value=0.2, max_value=0.5, step=0.1)))
    model.add(layers.Dense(hp.Int('dense2_units', min_value=16, max_value=64, step=16), activation='relu'))
    model.add(layers.Dropout(hp.Float('dropout3', min_value=0.2, max_value=0.5, step=0.1)))
    model.add(layers.Dense(3, activation='softmax'))

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model